### Imports
Import our libraries 

In [ ]:
#Basic imports

import pandas as pd
import numpy as np

# Building recommender systems

#!pip install surprise
from google.colab import files #To load data on Google Colab
import io
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, accuracy, accuracy, SVD, KNNWithMeans
from surprise.model_selection import cross_validate
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

### Data Loading

In [ ]:
#Load our data
uploaded = files.upload()

Saving movies.csv to movies (1).csv


In [ ]:
movies = pd.read_csv(io.BytesIO(uploaded['movies.csv']))
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
uploaded = files.upload()

Saving imdb_data.csv to imdb_data (1).csv


In [ ]:
imdb = pd.read_csv(io.BytesIO(uploaded['imdb_data.csv']))
imdb.head()

,movieId,title_cast,director,runtime,budget,plot_keywords
0,1,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


In [ ]:
#Merge the movies and imdb columns on movieId

df = pd.merge(movies,imdb, on = 'movieId')
df.head()

,movieId,title,genres,title_cast,director,runtime,budget,plot_keywords
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Jumanji (1995),Adventure|Children|Fantasy,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Grumpier Old Men (1995),Comedy|Romance,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Father of the Bride Part II (1995),Comedy,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


In [ ]:
#Drop the runtime and budget column

df.drop(columns=['runtime', 'budget'], axis=1, inplace=True)
df.head()

,movieId,title,genres,title_cast,director,plot_keywords
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,toy|rivalry|cowboy|cgi animation
1,2,Jumanji (1995),Adventure|Children|Fantasy,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,board game|adventurer|fight|game
2,3,Grumpier Old Men (1995),Comedy|Romance,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,boat|lake|neighbor|rivalry
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,black american|husband wife relationship|betra...
4,5,Father of the Bride Part II (1995),Comedy,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,fatherhood|doberman|dog|mansion


### Content-based Filtering

In [ ]:
#Ensure all datatypes are strings

cols = ['title_cast', 'plot_keywords', 'genres', 'director']
for col in cols:
  df[col] = df[col].astype(str)

#Concatenate the names in the director and title_cast columns

df.director = df.director.apply(lambda name: "".join(name.lower() for name in name.split()))
df.title_cast = df.title_cast.apply(lambda name: "".join(name.lower() for name in name.split()))

#Clean the rows of any special characters (|) and then fix the title cast column

df.title_cast = df.title_cast.map(lambda x: x.split('|')[:5])
df.title_cast = df.title_cast.apply(lambda x: " ".join(x))

#Clean the plot keywords the same way, retrieving the first five words again

df.plot_keywords= df.plot_keywords.map(lambda keyword: keyword.split('|')[:5])
df.plot_keywords = df.plot_keywords.apply(lambda keyword: " ".join(keyword))

#Cleaning the genres column

df.genres = df.genres.map(lambda word: word.lower().split('|'))
df.genres = df.genres.apply(lambda word: " ".join(word))


In [ ]:
#View the cleaned data

df.head()

,movieId,title,genres,title_cast,director,plot_keywords
0,1,Toy Story (1995),adventure animation children comedy fantasy,tomhanks timallen donrickles jimvarney wallace...,johnlasseter,toy rivalry cowboy cgi animation
1,2,Jumanji (1995),adventure children fantasy,robinwilliams jonathanhyde kirstendunst bradle...,jonathanhensleigh,board game adventurer fight game
2,3,Grumpier Old Men (1995),comedy romance,waltermatthau jacklemmon sophialoren ann-margr...,markstevenjohnson,boat lake neighbor rivalry
3,4,Waiting to Exhale (1995),comedy drama romance,whitneyhouston angelabassett lorettadevine lel...,terrymcmillan,black american husband wife relationship betra...
4,5,Father of the Bride Part II (1995),comedy,stevemartin dianekeaton martinshort kimberlywi...,alberthackett,fatherhood doberman dog mansion


In [ ]:
(len(df), len(df.movieId))

(24866, 24866)

In [ ]:
#Merge the columns for our vectorizer

df['word_bank'] = ''
word_bank = []
 
cols = ['title_cast', 'director', 'plot_keywords', 'genres']

#Generate the word_bank: ie. a list of words to feed into the vectorizer

for row in range(len(df)):
    string_ = ''
    for col in cols:
        string_ += df.iloc[row][col] + " "        
    word_bank.append(string_)


In [ ]:
#Append wordbank list as a column to dataframe

df['word_bank'] = word_bank

df.set_index('movieId', inplace=True)

#Drop the columns

df.drop(columns=['title_cast', 'director', 'plot_keywords', 'genres'], inplace=True)

In [ ]:
#View the data

df.head()

,title,word_bank
movieId,,
1,Toy Story (1995),tomhanks timallen donrickles jimvarney wallace...
2,Jumanji (1995),robinwilliams jonathanhyde kirstendunst bradle...
3,Grumpier Old Men (1995),waltermatthau jacklemmon sophialoren ann-margr...
4,Waiting to Exhale (1995),whitneyhouston angelabassett lorettadevine lel...
5,Father of the Bride Part II (1995),stevemartin dianekeaton martinshort kimberlywi...


In [ ]:
#Vectorizer the wordbank column

vectorizer = CountVectorizer()
vect_matrix = vectorizer.fit_transform(df.word_bank)

In [ ]:
#Find similarity between movies

cos_sim = cosine_similarity(vect_matrix, vect_matrix)

### Build the recommendation systems

In [ ]:
def movie_recommender(title, n=11):
   
    #Reset the index has been reset

    new_df = df.reset_index()
    
    # Extract the movie titles

    titles_list = new_df['title']

    #A 1-dimensional array with movie titles

    indices = pd.Series(new_df.index, index = df['title'])
    
    title_index = indices[title]
    
    #Get the similarity scores of the top n movies most similar to the user input

    sim_scores = list(enumerate(cos_sim[title_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n]
    
    # Exctract the titles of the top n most similar movies  
    movie_index = [i[0] for i in sim_scores]
    
    return titles_list.iloc[movie_index]


In [ ]:
movie_recommender('Toy Story (1995)')

3020                            Toy Story 2 (1999)
14758                           Toy Story 3 (2010)
22139    Toy Story Toons: Hawaiian Vacation (2011)
22140            Toy Story Toons: Small Fry (2011)
12927                         Open Season 2 (2008)
22919                 The Nutcracker Prince (1990)
20909                 Dug's Special Mission (2009)
13905                     Fantastic Mr. Fox (2009)
23079      Toy Story Toons: Partysaurus Rex (2012)
12503                          Space Chimps (2008)
Name: title, dtype: object

In [ ]:
movie_recommender('Waiting to Exhale (1995)')

23494                                     Melbourne (2014)
10323                                         Prime (2005)
21750                               One I Love, The (2014)
15374                        12 Storeys (Shier lou) (1997)
19471    Love Is All You Need (Den skaldede frisør) (2012)
13680                                 Kahlekuningas (2002)
18181                                   Being Flynn (2012)
19774                            Thanks for Sharing (2012)
23895                                  Opposite Day (2009)
16446                                     Submarine (2010)
Name: title, dtype: object

In [ ]:
movie_recommender('Ice Age (2002)')

10648           Ice Age 2: The Meltdown (2006)
13437    Ice Age: Dawn of the Dinosaurs (2009)
18129      Ice Age 4: Continental Drift (2012)
16662                   Winnie the Pooh (2011)
23854     A Flintstones Christmas Carol (1994)
9591             Pooh's Heffalump Movie (2005)
151                      Batman Forever (1995)
4873         Bill & Ted's Bogus Journey (1991)
7038                          Peter Pan (2003)
24597    Tom and Jerry: The Lost Dragon (2014)
Name: title, dtype: object

In [ ]:
movie_recommender('Jumanji (1995)')

10398                             Zathura (2005)
23498           The Gamers: Hands of Fate (2013)
540                     Super Mario Bros. (1993)
792                                Alaska (1996)
10247                          MirrorMask (2005)
13821           Where the Wild Things Are (2009)
3390                                 Hook (1991)
3343     Teenage Mutant Ninja Turtles III (1993)
12940        Gamers, The: Dorkness Rising (2008)
20162                       I Declare War (2012)
Name: title, dtype: object

In [ ]:
movie_recommender('Star Wars: The Clone Wars (2008)')

6220                        X2: X-Men United (2003)
4873              Bill & Ted's Bogus Journey (1991)
13424    Transformers: Revenge of the Fallen (2009)
151                           Batman Forever (1995)
7038                               Peter Pan (2003)
10871                       Superman Returns (2006)
16445                     X-Men: First Class (2011)
20341                     Great Expectations (2012)
16662                        Winnie the Pooh (2011)
13437         Ice Age: Dawn of the Dinosaurs (2009)
Name: title, dtype: object